Load data from recipe runs.

In [26]:
%use dataframe

val projectIds = DataFrame.read("Release-Train-Metro-Plan/src/main/kotlin/data/ProjectCoordinates.csv")
val dependencies = DataFrame.read("Release-Train-Metro-Plan/src/main/kotlin/data/DependenciesInUse.csv")

print("Loaded ${projectIds.rowsCount()} projects and ${dependencies.rowsCount()} dependencies.")

Loaded 60 projects and 965 dependencies.

In [7]:
import java.nio.file.Files
import java.nio.file.StandardOpenOption
import kotlin.io.path.Path
import kotlin.io.path.createFile

data class Artifact(val group: String?,
                   val artifact: String,
                   var parent: Artifact? = null) {
    override fun equals(other: Any?): Boolean = other is Artifact && other.group == group && other.artifact == artifact
    override fun hashCode(): Int = group.hashCode() * 31 + artifact.hashCode()
}

data class Repository(val path: String, val artifacts: Set<Artifact>,  val dependencies: Set<Artifact>) {
    override fun equals(other: Any?): Boolean = other is Repository && other.path == path
    override fun hashCode(): Int = path.hashCode()
}

val repos = mutableListOf<Repository>()

projectIds
    .select { it["repositoryPath", "repositoryBranch", "groupId", "artifactId"] }
    .groupBy { it["repositoryPath"]}
    .forEach {
        repos.add(Repository(it.key.repositoryPath, // repo by path
            it.group
                .map {
                    a -> Artifact(a.groupId, a.artifactId) }
                .toSet(), // artifacts contained in the repo
            dependencies
                .select { d-> d["repositoryPath", "repositoryBranch", "groupId", "artifactId"] }
                .filter { d -> d.repositoryBranch == "master" || d.repositoryBranch == "main" }
                .filter { d -> d.repositoryPath == it.key.repositoryPath  }
                .map { d -> Artifact(d.groupId, d.artifactId) }
                .toSet()
        ))
    }

println("derived ${repos.size} repositories from the data, containing ${repos.sumOf { it.artifacts.size }} artifacts and ${repos.sumOf { it.dependencies.size }} dependencies.")

derived 20 repositories from the data, containing 53 artifacts and 287 dependencies.


In [11]:
// all data as one
/*
println(
"""
@startuml
${repos.joinToString("\n") { it.asPlantUml() }}
${repos.joinToString("\n") { it.asParentRelationships() }}
@enduml
"""
)
*/

In [8]:

// print conenction between repos
enum class LinkType { parent, dependency }
data class Link(val src: String, val dist: String, val type: LinkType) {
    fun asD3() : String = "{ source: \"${src}\", target: \"${dist}\", type: \"${type}\" }"
}
data class Node(val id: String) {
    fun asD3() : String = "{ id: \"${id}\" }"
}

val edges = mutableSetOf<Link>()
for (repo in repos) {
    val parents = repo.artifacts
        .map { it.parent }
        .filterNotNull()

    repos
        .filter { it.artifacts.any { parents.contains(it) } }
        .filter { it.path != repo.path }
        .map { Link(repo.path, it.path, LinkType.parent) }
        .forEach { edges.add(it) }

    repo.dependencies
        .map { dep -> repos.find { it.artifacts.contains(dep) }?.path }
        .filterNotNull()
        .filter { it != repo.path }
        .map { Link(repo.path, it, LinkType.dependency) }
        .forEach { edges.add(it) }
}

val nodes = edges.map { listOf(it.src, it.dist) }.flatMap { it }.distinct().map { Node(it) }

println(nodes.joinToString(",\n\t", prefix = "export const nodes = [\n", postfix = "\n];") { it.asD3() })
println(edges.joinToString(",\n\t", prefix = "export const links = [\n", postfix = "\n];") { it.asD3() })


const nodes = [
{ id: "openrewrite/rewrite-all" },
	{ id: "openrewrite/rewrite-csharp" },
	{ id: "openrewrite/rewrite" },
	{ id: "openrewrite/rewrite-python" },
	{ id: "openrewrite/rewrite-build-gradle-plugin" },
	{ id: "openrewrite/rewrite-cucumber-jvm" },
	{ id: "openrewrite/rewrite-docker" },
	{ id: "openrewrite/rewrite-gitlab" },
	{ id: "openrewrite/rewrite-gradle-plugin" },
	{ id: "openrewrite/rewrite-gradle-tooling-model" },
	{ id: "openrewrite/rewrite-jackson" },
	{ id: "openrewrite/rewrite-java-dependencies" },
	{ id: "openrewrite/rewrite-jenkins" },
	{ id: "openrewrite/rewrite-migrate-java" },
	{ id: "openrewrite/rewrite-netty" },
	{ id: "openrewrite/rewrite-quarkus" },
	{ id: "openrewrite/rewrite-recipe-bom" },
	{ id: "openrewrite/rewrite-recipe-markdown-generator" },
	{ id: "openrewrite/rewrite-static-analysis" },
	{ id: "openrewrite/rewrite-struts" }
];
const links = [
{ source: "openrewrite/rewrite-all", target: "openrewrite/rewrite-csharp", type: "dependency" },
	{ source